In [ ]:
!pip install qiskit==1.4.2
!pip install qiskit-optimization
!pip install qiskit-algorithms
!pip install qiskit-ibm-runtime
!pip install docplex
!pip install cplex
!pip install qiskit-aer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.5/646.5 kB 31.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 20.9 MB/s eta 0:00:00
  Created wheel for docplex: filename=docplex-2.30.251-py3-none-any.whl size=685954 sha256=25b75cc8c28445da17f70b51d237f924dc359a46da0969205644b761441f19de
  Stored in directory: /root/.ca

In [ ]:
!git clone https://github.com/bimehta/WISER_Optimization_VG.git

Cloning into 'WISER_Optimization_VG'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 139 (delta 25), reused 112 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (139/139), 18.98 MiB | 22.57 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [ ]:

from __future__ import annotations
import logging
from itertools import combinations


In [ ]:
import sys
sys.path.append('/content/WISER_Optimization_VG/src/')

In [ ]:
from pathlib import Path
ROOT = Path('/content/WISER_Optimization_VG/')

In [ ]:
lp_file_path = ROOT / 'data/31bonds/docplex-bin-avgonly-nocplexvars.lp'
print(f"Loading data from: {lp_file_path}")

Loading data from: /content/WISER_Optimization_VG/data/31bonds/docplex-bin-avgonly-nocplexvars.lp


In [ ]:
!pip install docplex
!pip install cplex

In [ ]:
import pickle

with open('/content/exp0.pkl', 'rb') as f:
    data = pickle.load(f)

print("Keys found in the .pkl file:")
print(list(data.keys()))

Keys found in the .pkl file:
['experiment_id', 'ansatz', 'ansatz_params', 'theta_initial', 'device', 'optimizer', 'alpha', 'lp_file', 'shots', 'refx', 'refvalue', 'classical_hw', 'step3_time', 'step4_time', 'step3_fx_evals', 'step3_result_success', 'step3_result_message', 'step3_result_best_x', 'step3_result_best_fx', 'step4_num_epochs', 'step4_fx_evals', 'step4_result_best_x', 'step4_result_best_fx', 'step3_monitor_iter_thetas', 'step3_monitor_iter_gtheta', 'step3_monitor_iter_best_fx', 'step3_monitor_calls_thetas', 'step3_monitor_calls_gtheta', 'step4_monitor_iter_best_fx', 'step3_job_ids', 'step2_isa_ansatz']


In [ ]:
import pickle
import numpy as np
from pathlib import Path
import sys
import docplex.mp.model_reader

print("--- Setting up paths and imports ---")
ROOT = Path.cwd()
PROJECT_ROOT = ROOT / 'WISER_Optimization_VG'

sys.path.append(str(ROOT))
sys.path.append(str(PROJECT_ROOT / 'src'))
sys.path.append(str(PROJECT_ROOT / '_experiments'))

from optimization9 import uncompress_x
from local_search_py_new5 import variable_neighborhood_search
from doe_localsearch_new1 import doe_localsearch
from step_1_py_new3 import model_to_obj
print("--- Configuring the test ---")
doe = doe_localsearch['vns']
input_data_file = ROOT / 'exp0.pkl'
lp_file = PROJECT_ROOT / 'data/1/31bonds/docplex-bin-avgonly-nocplexvars.lp'
print(f"--- Loading model from {lp_file} to build objective function ---")
model = docplex.mp.model_reader.ModelReader.read(str(lp_file))
obj_fn = model_to_obj(model)
num_vars = model.number_of_binary_variables
print("Objective function created successfully.")
print(f"--- Loading data from {input_data_file} ---")
with open(input_data_file, 'rb') as f:
    data = pickle.load(f)

initial_x_compressed = data['step3_result_best_x']
initial_val = data['step3_result_best_fx']
print(f"Found one candidate solution with initial value: {initial_val:.4f}")
print("--- Running Variable Neighborhood Search on the candidate ---")
initial_x = initial_x_compressed
polished_x, polished_val, _, _, _ = variable_neighborhood_search(
    initial_x, initial_val, obj_fn, doe | {'refval': data['refvalue']}
)
print(f"  -> Polished value found: {polished_val:.4f}")
print("\n--- Saving final result ---")
output_directory = ROOT / 'my_results'
output_directory.mkdir(exist_ok=True)
output_file_path = output_directory / 'polished_result.pkl'

final_result = {
    'best_solution_bitstring': polished_x,
    'best_solution_value': polished_val,
    'original_value': initial_val,
    'original_file': str(input_data_file)
}

with open(output_file_path, 'wb') as f:
    pickle.dump(final_result, f)

print(f"✅ Success! The single best result was saved to: {output_file_path}")

--- Setting up paths and imports ---
--- Configuring the test ---
--- Loading model from /content/WISER_Optimization_VG/data/1/31bonds/docplex-bin-avgonly-nocplexvars.lp to build objective function ---
31 15
Objective function created successfully.
--- Loading data from /content/exp0.pkl ---
Found one candidate solution with initial value: 40.3934
--- Running Variable Neighborhood Search on the candidate ---
  -> Polished value found: 40.3047

--- Saving final result ---
✅ Success! The single best result was saved to: /content/my_results/polished_result.pkl


In [ ]:
import pickle
import pandas as pd
file_path = '/content/my_results/polished_result.pkl'

with open(file_path, 'rb') as f:
    results = pickle.load(f)

df = pd.DataFrame([results])
print(df)

                             best_solution_bitstring  best_solution_value  \
0  [0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, ...            40.304697   

   original_value      original_file  
0       40.393389  /content/exp0.pkl  


Step 4: Post-Processing